In [1]:
def reverse_lines(text):
    """
    Takes a text divided into lines and returns the lines in reversed order.
    
    Args:
        text (str): Input text with multiple lines
        
    Returns:
        str: Text with lines in reversed order
    """
    lines = text.split('\n')
    reversed_lines = lines[::-1]
    return '\n'.join(reversed_lines)

In [3]:
reversed = reverse_lines(
    """
    qc.cx(2, 9)
    qc.cx(0, 10)
    qc.cx(1, 11)
    
    qc.x(6)
    qc.x(7)
    qc.x(8)
    
    qc.cx(4, 9)
    qc.cx(5, 10)
    qc.cx(3, 11)
    
    qc.x(9)
    qc.x(10)
    qc.x(11)
    
    qc.cx(6, 12)
    qc.cx(7, 13)
    qc.cx(8, 14)
    
    qc.x(6)
    qc.x(7)
    qc.x(8)
    
    # reset to |0> qubit 6, 7, 8
    qc.reset(6)
    qc.reset(7)
    qc.reset(8)
    
    qc.mcx([6, 12], 15)
    qc.mcx([7, 13], 16)
    qc.mcx([8, 14], 17)
    
    qc.x(6)
    qc.x(7)
    qc.x(8)
    
    qc.reset(11)
    qc.reset(12)
    qc.reset(13)
    qc.reset(14)
    
    qc.cx(15, 6)
    qc.cx(16, 7)
    qc.cx(17, 8)
    
    qc.mcx([0, 6], 18)
    qc.cx(18, 0)
    qc.cx(0, 9)
    qc.reset(18)
    
    qc.mcx([1, 6], 18)
    qc.cx(18, 1)
    qc.cx(1, 10)
    qc.reset(18)
    
    qc.mcx([2, 7], 18)
    qc.cx(18, 2)
    qc.cx(2, 8)
    qc.reset(18)
    
    qc.mcx([3, 7], 18)
    qc.cx(18, 3)
    qc.cx(3, 11)
    qc.reset(18)
    
    qc.mcx([4, 8], 18)
    
    qc.x(11)
    qc.cx(18, 4)
    qc.reset(18)
    qc.cx(4, 9)
    qc.cx(11, 14)
    
    qc.x(9)
    qc.x(11)
    
    qc.mcx([5, 8], 18)
    qc.cx(18, 5)
    
    qc.cx(9, 12)
    qc.reset(18)
    
    qc.mcx([6, 7, 8], 19)
    qc.p(beta, 19)
    
    qc.cx(5, 10)
    qc.x(9)
    qc.x(10)
    qc.cx(10, 13)
    qc.x(10)
    """
)

In [5]:
print(reversed)

    
    qc.x(10)
    qc.cx(10, 13)
    qc.x(10)
    qc.x(9)
    qc.cx(5, 10)
    
    qc.p(beta, 19)
    qc.mcx([6, 7, 8], 19)
    
    qc.reset(18)
    qc.cx(9, 12)
    
    qc.cx(18, 5)
    qc.mcx([5, 8], 18)
    
    qc.x(11)
    qc.x(9)
    
    qc.cx(11, 14)
    qc.cx(4, 9)
    qc.reset(18)
    qc.cx(18, 4)
    qc.x(11)
    
    qc.mcx([4, 8], 18)
    
    qc.reset(18)
    qc.cx(3, 11)
    qc.cx(18, 3)
    qc.mcx([3, 7], 18)
    
    qc.reset(18)
    qc.cx(2, 8)
    qc.cx(18, 2)
    qc.mcx([2, 7], 18)
    
    qc.reset(18)
    qc.cx(1, 10)
    qc.cx(18, 1)
    qc.mcx([1, 6], 18)
    
    qc.reset(18)
    qc.cx(0, 9)
    qc.cx(18, 0)
    qc.mcx([0, 6], 18)
    
    qc.cx(17, 8)
    qc.cx(16, 7)
    qc.cx(15, 6)
    
    qc.reset(14)
    qc.reset(13)
    qc.reset(12)
    qc.reset(11)
    
    qc.x(8)
    qc.x(7)
    qc.x(6)
    
    qc.mcx([8, 14], 17)
    qc.mcx([7, 13], 16)
    qc.mcx([6, 12], 15)
    
    qc.reset(8)
    qc.reset(7)
    qc.reset(6)
    # reset to |0> qubit 6, 7,